In [1]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import tensorflow as tf
from bayes_opt import BayesianOptimization

2025-01-30 11:43:02.025896: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
#creating the overall dictionary: the one representing the nutrition label facts. I will copy this dictionary for any 
#ingredient I use.

overall_dict = {}

overall_dict["Fat"] = 0
overall_dict["Saturated fatty acids"] = 0
overall_dict["Fatty acids, total trans"] = 0
overall_dict["Cholesterol"] = 0
overall_dict["Sodium"] = 0
overall_dict["Carbohydrate"] = 0
overall_dict["Fiber"] = 0
overall_dict["Sugars"] = 0
overall_dict["Protein"] = 0
overall_dict["Calcium"] = 0
overall_dict["Iron"] = 0
overall_dict["Potassium"] = 0
overall_dict["Vitamin D"] = 0
overall_dict["Weight"] = 0

In [6]:
#nutritional information for tates cookies per 10 grams (each number was initially for  28 grams. So I had to multiply 
#each number by (10 * (1 / 28)) to get the information for 10 grams)

tates_dict = overall_dict.copy()
tates_dict["Fat"] = 7
tates_dict["Saturated fatty acids"] = 4.5
tates_dict["Fatty acids, total trans"] = 0
tates_dict["Cholesterol"] = .025
tates_dict["Sodium"] = .16
tates_dict["Carbohydrate"] = 18
tates_dict["Fiber"] = .8
tates_dict["Sugars"] = 12
tates_dict["Protein"] = 2
tates_dict["Calcium"] = .0000001
tates_dict["Iron"] = .0009
tates_dict["Potassium"] = .01
tates_dict["Vitamin D"] = .05
tates_dict["Weight"] = 28

In [9]:
ingredient_dict = overall_dict.copy()

def reader(file,ingredient_dict,desired_serving):
    with open(file, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        for i,row in enumerate(csvreader):
            if len(row) > 0 and row[0] in ingredient_dict:
                if row[2] == "mg":
                    ingredient_dict[row[0]] = (float(row[1]) / 1000)
                elif row[2] == "mcg":
                    ingredient_dict[row[0]] = (float(row[1]) / 1000000)
                else: #grams
                    ingredient_dict[row[0]] = (float(row[1]))
            if i == 4:
                index = row.index("g")
                ingredient_dict["Weight"] = float(row[index-1])
    return ingredient_dict

In [15]:
#the various ingredients in Tate's. Transforming the online nutrritional information of each ingredient into individual dictionaries.
desired_serving = 1
semi_sweet_chocolate_file = "semisweet_chocolate_chips_by_raleys.csv"
semi_sweet_chocolate_dict = overall_dict.copy()
reader(semi_sweet_chocolate_file,semi_sweet_chocolate_dict,desired_serving)

unbleached_flour_file = "flour_unbleached_enriched_allpurpose_wheat.csv"
unbleached_flour_dict = overall_dict.copy()
reader(unbleached_flour_file,unbleached_flour_dict,desired_serving)

salted_butter_file = "butter_salted.csv"
salted_butter_dict = overall_dict.copy()
reader(salted_butter_file,salted_butter_dict,desired_serving)

cane_sugar_file = "granulated_pure_cane_sugar.csv"
cane_sugar_dict = overall_dict.copy()
reader(cane_sugar_file,cane_sugar_dict,desired_serving)

brown_cane_sugar_file = "brown_sugar_cane_by_frusecha.csv"
brown_cane_sugar_dict = overall_dict.copy()
reader(brown_cane_sugar_file,brown_cane_sugar_dict,desired_serving)

eggs_file = "egg_fresh_raw_whole.csv"
eggs_dict = overall_dict.copy()
reader(eggs_file,eggs_dict,desired_serving)

baking_soda_file = "leavening_agents_baking_soda.csv"
baking_soda_dict = overall_dict.copy()
reader(baking_soda_file,baking_soda_dict,desired_serving)

salt_file = "salt_table.csv"
salt_dict = overall_dict.copy()
reader(salt_file,salt_dict,desired_serving)

natural_vanilla_flavor_file = "vanilla_flavoring_syrup_by_r_torre__coinc.csv"
natural_vanilla_flavor_dict = overall_dict.copy()
reader(natural_vanilla_flavor_file,natural_vanilla_flavor_dict,desired_serving)

vanilla_extract_file = "vanilla_extract.csv"
vanilla_extract_dict = overall_dict.copy()
reader(vanilla_extract_file,vanilla_extract_dict,desired_serving)

dictionary_list = [semi_sweet_chocolate_dict,unbleached_flour_dict,salted_butter_dict,cane_sugar_dict,brown_cane_sugar_dict,eggs_dict,baking_soda_dict,salt_dict,vanilla_extract_dict]
heavy_dictionary_list = [semi_sweet_chocolate_dict,unbleached_flour_dict,salted_butter_dict,cane_sugar_dict,brown_cane_sugar_dict,eggs_dict]
key_list = ["Fat","Saturated fatty acids","Fatty acids, total trans","Cholesterol","Sodium","Carbohydrate","Fiber","Sugars","Protein","Calcium","Iron","Potassium","Vitamin D","Weight"]
key_list_sodium = ["Fat","Saturated fatty acids","Fatty acids, total trans","Cholesterol","Carbohydrate","Fiber","Sugars","Protein","Calcium","Iron","Potassium","Vitamin D","Weight"]

In [16]:
def building_x_list(number,dictionary_list):
    x_list = []
    for variable in range(0, len(dictionary_list)):
        x_list.append(number)
    return x_list

x_list = building_x_list(1,dictionary_list)

In [17]:
def equations(dictionary_list,x_list,desired_dict,key_list):
    list_of_equations = []
    for key in key_list:
        equation = 0
        for i,dictionary in enumerate(dictionary_list):
            equation = equation + (dictionary[key] * (x_list[i]))
        #subtracting the constant that the equation equals (a0x0 + a1x1 + ... + a13x13 = C1 -->  a0x0 + a1x1 + ... + a13x13 - C1)
        list_of_equations.append(equation-(desired_dict[key]*desired_serving))
    return list_of_equations

In [40]:
semi_sweet_chocolate_dict


{'Fat': 2.667,
 'Saturated fatty acids': 1.667,
 'Fatty acids, total trans': 0.0,
 'Cholesterol': 0.0,
 'Sodium': 0.0,
 'Carbohydrate': 6.67,
 'Fiber': 0.67,
 'Sugars': 5.33,
 'Protein': 0.67,
 'Calcium': 0.0,
 'Iron': 0.00027,
 'Potassium': 0.0,
 'Vitamin D': 0,
 'Weight': 10.0}

In [38]:
equations(heavy_dictionary_list,[2.3620467, 0.29056025, 0.08481605, 0, 0, 0.05550218],tates_dict,key_list)

[0.0833070206299995,
 -0.10939591990999986,
 0.028041673120000003,
 -0.021111773829,
 -0.153641972929,
 -0.11351416790000357,
 0.7825712890000001,
 0.5927771587000006,
 0.04077140779999988,
 0.001124447253,
 -9.590948539999985e-05,
 -0.005107948021,
 -0.049999988899564,
 -0.0707482000000006]

In [48]:
def initialize_variables_tf(n):
    # Random positive values 
    initial_values = tf.random.uniform(shape=(n,), minval=0.0, maxval=(1.8))
    
    # Sort in descending order
    initial_values = tf.sort(initial_values, direction='DESCENDING')
    
    # Normalize to sum to 1
    #initial_values = initial_values / tf.reduce_sum(initial_values)
    
    return tf.Variable(initial_values, trainable=True)

In [49]:
def loss_tf(predicted):
    
    return tf.reduce_mean(tf.square(predicted))

In [54]:
def running_tf(dictionary_list,target_dict):  
    n = len(dictionary_list)
    variables_tf = initialize_variables_tf(n)
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

# Training loop
    for epoch in range(400):  # Number of epochs
        with tf.GradientTape() as tape:
            
        # Step 1: Evaluate the equations
            predictions1 = equations(dictionary_list,variables_tf,target_dict,key_list)

            predictions = tf.convert_to_tensor(predictions1)

        # Step 2: Compute the loss
            loss = loss_tf(predictions)

    # Step 3: Compute gradients
        gradients = tape.gradient(loss, [variables_tf])

    # Step 4: Apply gradients to update variables
        optimizer.apply_gradients(zip(gradients, [variables_tf]))

    # Apply constraints
    # 1. Ensure all variables are positive
        variables_tf.assign(tf.maximum(variables_tf, 0.0))
    
    # 2. Ensure variables are in descending order
        #variables_tf.assign(tf.sort(variables_tf, direction='DESCENDING'))

        if epoch % 20 == 0:  # Print every 20 epochs
            print(f"Epoch {epoch}, Loss: {loss.numpy():.4f}, Variables: {variables_tf.numpy()}")

    x_for_one_serving = []
    for x in variables_tf.numpy():
        x_for_one_serving.append(x * 270)
    print(x_for_one_serving)

    return (variables_tf.numpy())

In [55]:
running_tf(heavy_dictionary_list,tates_dict)

Epoch 0, Loss: 53.2514, Variables: [0.86856186 0.7961991  0.37248674 0.09400445 0.         0.1317291 ]
Epoch 20, Loss: 0.0708, Variables: [1.050708   0.6972396  0.45294484 0.40074867 0.22630641 0.00644752]
Epoch 40, Loss: 0.0326, Variables: [1.0676028  0.630691   0.47778893 0.42146996 0.22819506 0.        ]
Epoch 60, Loss: 0.0276, Variables: [1.0786926  0.6051593  0.48196617 0.43051597 0.22490819 0.        ]
Epoch 80, Loss: 0.0263, Variables: [1.0882632  0.5953169  0.48191208 0.43449643 0.21935199 0.        ]
Epoch 100, Loss: 0.0256, Variables: [1.0971568  0.59127057 0.48036462 0.43659166 0.21303512 0.        ]
Epoch 120, Loss: 0.0249, Variables: [1.1056828  0.58937085 0.47830105 0.43798694 0.20652089 0.        ]
Epoch 140, Loss: 0.0244, Variables: [1.1139582  0.5882726  0.47608042 0.43911925 0.20001535 0.        ]
Epoch 160, Loss: 0.0238, Variables: [1.1220281  0.58747977 0.47383448 0.44014966 0.19359271 0.        ]
Epoch 180, Loss: 0.0232, Variables: [1.1299117  0.5868093  0.47161093

array([1.2055902 , 0.5810498 , 0.4501373 , 0.451266  , 0.12553309,
       0.        ], dtype=float32)